In [ ]:
!pip install nilearn
!pip install chart_studio
!pip install -U kaleido
!git clone https://github.com/thanhvd18/plot-cortical-thickness-matlab.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
fatal: destination path 'plot-cortical-thickness-matlab' already exists and is not an empty directory.


In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from nilearn import datasets, surface
import pandas as pd
import chart_studio.plotly as py

def region2surface(P, pathToAnnotationRh):
    """
    Convert region weight to surface weight based on annotation data.

    Parameters:
        P (ndarray): Region weights.
        pathToAnnotationRh (str): Path to the annotation file for the right hemisphere.

    Returns:
        ndarray: Surface weight data.
    """
    from nibabel import freesurfer as nfs
    rh = nfs.read_annot(pathToAnnotationRh)
    P = P.ravel()
    atlas_left_cdata = rh[0]  # Left hemisphere annotation data
    atlas_left_id = rh[1][:, 4]  # Left hemisphere region IDs

    w_left_cdata = np.zeros(atlas_left_cdata.shape)
    w_right_cdata = np.zeros(atlas_left_cdata.shape)

    for i in range(int(len(P)/2)):
        w_left_cdata[atlas_left_cdata == i] = P[2*i]  # Assign weight to corresponding regions in the left hemisphere
        w_right_cdata[atlas_left_cdata == i] = P[2*i+1]  # Assign weight to corresponding regions in the right hemisphere

    surface_data = np.hstack([w_left_cdata, w_right_cdata])  # Combine left and right hemisphere surface data
    return surface_data

def vertex2color(gg_mesh, is_normalize=True, colormap=['#08519c', '#3182bd', '#6baed6', '#bdd7e7', '#eff3ff', '#CCCCCC', '#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15']):
    """
    Convert surface mesh weights to vertex colors based on a colormap.

    Parameters:
        gg_mesh (ndarray): Surface mesh weights.
        is_normalize (bool): Flag indicating whether to normalize the weights.
        colormap (list): List of colors in the colormap.

    Returns:
        list: Vertex colors.
    """
    if is_normalize:
        min_val = -1
        max_val = 1
    else:
        min_val = np.min(gg_mesh)
        max_val = np.max(gg_mesh)

    range_ = max_val - min_val
    step = range_ / (len(colormap) - 1)
    vertex_color = [colormap[int((x + 1e-6) / step)] for x in (gg_mesh.ravel())]  # Determine color for each vertex based on weight
    return vertex_color

def show_brain_space(surface_weight):
    """
    Display the brain surface with color-coded weights.

    Parameters:
        surface_weight (list): List of vertex colors representing the surface weights.

    Returns:
        plotly.graph_objs._figure.Figure: The figure object displaying the brain surface.
    """
    # Initialize figure with 3D subplot
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])

    fig.add_trace(
        go.Mesh3d(x=x, y=y, z=z, i=i, j=j, k=k, vertexcolor=surface_weight, showscale=True),  # Create 3D mesh plot with surface weights
        row=1, col=1
    )
    return fig




In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from nilearn import datasets, surface
import pandas as pd
import chart_studio.plotly as py

# Read surface data
vertices, triangles = surface.load_surf_mesh('plot-cortical-thickness-matlab/brainspace/mesh.inflated.freesurfer.gii')
# Read annotation of atlas
pathToAnnotationRh = 'plot-cortical-thickness-matlab/brainspace/lh.Schaefer2018_200Parcels_7Networks_order.annot'

# Read weight data
df = pd.read_csv('plot-cortical-thickness-matlab/data/mean.layer.0.csv')
display(df.head())
region_weight = df.x

,Unnamed: 0,x
0,V1,-0.051002
1,V2,-0.095142
2,V3,-0.113318
3,V4,-0.131156
4,V5,0.024191


In [ ]:
# Convert vertices and triangles into coordinate X, Y, Z for plotting
x, y, z = vertices.T
i, j, k = np.asarray(triangles).T

# Map region weights to the brain surface
surface_weight = region2surface(region_weight, pathToAnnotationRh)

# colormap = ['#08519c', '#3182bd', '#6baed6', '#CCCCCC', '#CCCCCC','#CCCCCC','#CCCCCC', '#CCCCCC', '#fb6a4a', '#de2d26', '#a50f15']
colormap = ['#08519c', '#3182bd', '#CCCCCC', '#CCCCCC', '#CCCCCC','#CCCCCC','#CCCCCC', '#CCCCCC', '#CCCCCC', '#de2d26', '#a50f15']

# Change color map based on the surface weights
surface_weight = vertex2color(surface_weight,is_normalize=False)

# Create a 3D plot of the brain surface with the weight-based colors
fig = show_brain_space(surface_weight)

In [ ]:
import plotly
from google.colab import files

plotly.offline.plot(fig, filename='result.html')
files.download('result.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import IPython
# File size exceeds display limit. Please download the file for viewing interactive result.
IPython.display.HTML(filename='result.html')
